## Deliverable 2. Create a Customer Travel Destinations Map.

In [45]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [46]:
# 1. Import the WeatherPy_database.csv file.
city_data_df = pd.read_csv("C:\\Users\\hp\\Class\\challenge\\Weather_Database\\WeatherPy_Database.csv")
city_data_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,2022-09-08 02:06:13,-0.7393,-90.3518,68.56,94,100,5.99,overcast clouds
1,1,Qaanaaq,GL,2022-09-08 02:06:58,77.4840,-69.3632,28.89,80,51,3.04,broken clouds
2,2,Punta Arenas,CL,2022-09-08 02:06:01,-53.1500,-70.9167,35.71,80,40,12.66,scattered clouds
3,3,Jamestown,US,2022-09-08 02:05:45,42.0970,-79.2353,65.17,70,100,3.44,overcast clouds
4,4,Waingapu,ID,2022-09-08 02:15:49,-9.6567,120.2641,85.59,61,40,11.70,scattered clouds


In [47]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [48]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Waingapu,ID,2022-09-08 02:15:49,-9.6567,120.2641,85.59,61,40,11.70,scattered clouds
8,8,Tulum,MX,2022-09-08 02:15:52,20.2167,-87.4667,79.30,84,31,4.18,scattered clouds
14,14,Pokhara,NP,2022-09-08 02:15:57,28.2333,83.9833,75.02,81,35,0.16,scattered clouds
15,15,Rikitea,PF,2022-09-08 02:06:54,-23.1203,-134.9692,72.59,75,0,23.58,clear sky
16,16,Tabuk,PH,2022-09-08 02:15:59,17.4189,121.4443,85.06,73,49,4.56,scattered clouds


In [49]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

305


City_ID                305
City                   305
Country                305
Date                   305
Lat                    305
Lng                    305
Max Temp               305
Humidity               305
Cloudiness             305
Wind Speed             305
Current Description    305
dtype: int64

In [50]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

305


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Waingapu,ID,2022-09-08 02:15:49,-9.6567,120.2641,85.59,61,40,11.70,scattered clouds
8,8,Tulum,MX,2022-09-08 02:15:52,20.2167,-87.4667,79.30,84,31,4.18,scattered clouds
14,14,Pokhara,NP,2022-09-08 02:15:57,28.2333,83.9833,75.02,81,35,0.16,scattered clouds
15,15,Rikitea,PF,2022-09-08 02:06:54,-23.1203,-134.9692,72.59,75,0,23.58,clear sky
16,16,Tabuk,PH,2022-09-08 02:15:59,17.4189,121.4443,85.06,73,49,4.56,scattered clouds


In [51]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Waingapu,ID,85.59,scattered clouds,-9.6567,120.2641,
8,Tulum,MX,79.30,scattered clouds,20.2167,-87.4667,
14,Pokhara,NP,75.02,scattered clouds,28.2333,83.9833,
15,Rikitea,PF,72.59,clear sky,-23.1203,-134.9692,
16,Tabuk,PH,85.06,scattered clouds,17.4189,121.4443,
17,Bengkulu,ID,79.00,overcast clouds,-3.8004,102.2655,
18,Fare,PF,77.83,light rain,-16.7000,-151.0167,
19,Flinders,AU,70.18,overcast clouds,-34.5833,150.8552,
21,Hithadhoo,MV,81.97,broken clouds,-0.6000,73.0833,
22,Gushi,CN,73.27,overcast clouds,34.6425,109.5843,


In [52]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
print("complete!")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
complete!


In [53]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Waingapu,ID,85.59,scattered clouds,-9.6567,120.2641,Pondok Wisata Elim
8,Tulum,MX,79.30,scattered clouds,20.2167,-87.4667,Kin Ha Tulum
14,Pokhara,NP,75.02,scattered clouds,28.2333,83.9833,OYO 170 Hotel View Point
15,Rikitea,PF,72.59,clear sky,-23.1203,-134.9692,People ThankYou
16,Tabuk,PH,85.06,scattered clouds,17.4189,121.4443,Golden Berries Hotel and Convention
17,Bengkulu,ID,79.00,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
18,Fare,PF,77.83,light rain,-16.7000,-151.0167,Maitai Lapita Village Huahine
19,Flinders,AU,70.18,overcast clouds,-34.5833,150.8552,Shellharbour Resort & Conference Centre
21,Hithadhoo,MV,81.97,broken clouds,-0.6000,73.0833,Scoop Guest House
22,Gushi,CN,73.27,overcast clouds,34.6425,109.5843,Wansheng Yuan Hotel


In [54]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))